In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
import math
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
import torch
import glob

In [2]:
import pandas as pd

# Read the play_data and tackles_data from CSV files
play_data = pd.read_csv('data/plays.csv')
tackles_data = pd.read_csv('data/tackles.csv')

# Group play_data by 'gameId'
grouped_play_data = play_data.groupby('gameId')

# Initialize an empty dictionary to store DataFrames for each game
game_data_dict = {}

# Iterate through each group (each unique gameId)
for game_id, play_group in grouped_play_data:
    # Filter the tackles data for the current 'gameId'
    tackles_for_game = tackles_data[tackles_data['gameId'] == game_id]
    
    # Get the playIds corresponding to tackles in this game
    tackle_playIds = tackles_for_game['playId'].unique()
    
    # Filter the play_group to keep only playIds corresponding to tackles
    filtered_play_group = play_group[play_group['playId'].isin(tackle_playIds)]
    
    # Store the filtered play data for the current game in the dictionary
    game_data_dict[game_id] = filtered_play_group

# Now, 'game_data_dict' contains DataFrames for each unique 'gameId' with only tackle plays


In [3]:
# Assuming you have already created the 'game_data_dict' dictionary as mentioned before

# Specify the 'gameId' of the game you want to print tackles for
target_game_id = 2022090800  # Replace with the actual 'gameId' you're interested in

# Check if the 'gameId' is in the dictionary
if target_game_id in game_data_dict:
    # Get the DataFrame for the specified game
    game_df = game_data_dict[target_game_id]
    
    # Print the DataFrame containing the tackles for the game
    print(game_df)
else:
    print(f"Game with 'gameId' {target_game_id} not found in the dictionary.")


           gameId  playId  ballCarrierId ballCarrierDisplayName  \
209    2022090800    2184          42489           Stefon Diggs   
262    2022090800    2307          44881            Cooper Kupp   
578    2022090800    3304          52494              Zack Moss   
1131   2022090800    2648          46076             Josh Allen   
1401   2022090800    2336          47853      Darrell Henderson   
...           ...     ...            ...                    ...   
11576  2022090800     393          47853      Darrell Henderson   
11580  2022090800    3431          53678          Ben Skowronek   
11953  2022090800     646          47879            Dawson Knox   
12048  2022090800    1286          44881            Cooper Kupp   
12206  2022090800     896          54528             James Cook   

                                         playDescription  quarter  down  \
209    (9:24) (Shotgun) J.Allen pass short left to S....        3     1   
262    (7:28) (Shotgun) M.Stafford pass deep 

In [4]:
# Filter the tackles data to include plays with at least one tackle
plays_with_tackle = tackles_data[tackles_data['tackle'] == 1]

# Filter the tackles data to include plays with only pff_missedTackle (no tackles)
plays_with_missed_tackle = tackles_data[(tackles_data['pff_missedTackle'] == 1) & (tackles_data['tackle'] == 0)]

# Count the unique 'playId' values in each filtered data
count_plays_with_tackle = len(plays_with_tackle['playId'].unique())
count_plays_with_missed_tackle = len(plays_with_missed_tackle['playId'].unique())

# Print the counts
print(f"Number of plays with at least one tackle: {count_plays_with_tackle}")
print(f"Number of plays with only pff_missedTackle (no tackles): {count_plays_with_missed_tackle}")

Number of plays with at least one tackle: 3776
Number of plays with only pff_missedTackle (no tackles): 1423


In [5]:
# make dataframes for each tracking_week_x.csv
path = 'data/tracking_week_*.csv'
all_files = glob.glob(path)
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
tracking_data = pd.concat(li, axis=0, ignore_index=True)


In [6]:
print(tracking_data.head())

       gameId  playId    nflId     displayName  frameId  \
0  2022090800      56  35472.0  Rodger Saffold        1   
1  2022090800      56  35472.0  Rodger Saffold        2   
2  2022090800      56  35472.0  Rodger Saffold        3   
3  2022090800      56  35472.0  Rodger Saffold        4   
4  2022090800      56  35472.0  Rodger Saffold        5   

                         time  jerseyNumber club playDirection      x      y  \
0  2022-09-08 20:24:05.200000          76.0  BUF          left  88.37  27.27   
1  2022-09-08 20:24:05.299999          76.0  BUF          left  88.47  27.13   
2  2022-09-08 20:24:05.400000          76.0  BUF          left  88.56  27.01   
3  2022-09-08 20:24:05.500000          76.0  BUF          left  88.64  26.90   
4  2022-09-08 20:24:05.599999          76.0  BUF          left  88.72  26.80   

      s     a   dis       o     dir         event  
0  1.62  1.15  0.16  231.74  147.90           NaN  
1  1.67  0.61  0.17  230.98  148.53  pass_arrived  
2  1.57 

In [7]:
# Load the tracking data for week 1 into a DataFrame
tracking_data = pd.read_csv('data/tracking_week_1.csv')

# Filter the tracking data for nflId 35472
player_data = tracking_data[tracking_data['nflId'] == 35472]

# Convert the timestamp to a datetime object
player_data['time'] = pd.to_datetime(player_data['time'])

# Define the game start time for week 1 (replace with the actual start time)
game_start_time = pd.Timestamp(str(player_data['time'].iloc[0]))

# Calculate the game time for each tracking data point
player_data['game_time'] = game_start_time + (player_data['time'] - player_data['time'].iloc[0])

# Now, the 'game_time' column in player_data represents the game time
print(player_data[['time', 'game_time']])

                            time                  game_time
0     2022-09-08 20:24:05.200000 2022-09-08 20:24:05.200000
1     2022-09-08 20:24:05.299999 2022-09-08 20:24:05.299999
2     2022-09-08 20:24:05.400000 2022-09-08 20:24:05.400000
3     2022-09-08 20:24:05.500000 2022-09-08 20:24:05.500000
4     2022-09-08 20:24:05.599999 2022-09-08 20:24:05.599999
...                          ...                        ...
70290 2022-09-08 22:53:34.599999 2022-09-08 22:53:34.599999
70291 2022-09-08 22:53:34.700000 2022-09-08 22:53:34.700000
70292 2022-09-08 22:53:34.799999 2022-09-08 22:53:34.799999
70293 2022-09-08 22:53:34.900000 2022-09-08 22:53:34.900000
70294 2022-09-08 22:53:35.000000 2022-09-08 22:53:35.000000

[1698 rows x 2 columns]


In [8]:
import pandas as pd

# Assuming you have already loaded the tackles data into a DataFrame named 'tackles_data'
tackles_data = pd.read_csv('data/tackles.csv')
# Filter the tackles data to include plays with tackles or missed tackles
tackle_missed_tackle_data = tackles_data[(tackles_data['tackle'] == 1) | (tackles_data['pff_missedTackle'] == 1)]

# Group the filtered data by 'gameId' and 'playId' and count the number of unique occurrences
unique_combinations = tackle_missed_tackle_data.groupby(['gameId', 'playId']).size().reset_index(name='count')

# Filter the unique combinations to include only those where count is greater than 1
unique_combinations = unique_combinations[unique_combinations['count'] > 1]

# Merge the filtered combinations with the original tackles data to get the desired rows
result = pd.merge(unique_combinations, tackles_data, on=['gameId', 'playId'], how='inner')

# Now, 'result' contains every tackle where gameId and playId combinations are unique
# and there is either a tackle=1 or pff_missedTackle=1 (excluding plays with only assist=1)
print(len(result))
#total missed tackles
total_missed_tackles = len(result[result['pff_missedTackle'] == 1])
print(f"Total missed tackles: {total_missed_tackles}")
#total tackles
total_tackles = len(result[result['tackle'] == 1])
print(f"Total tackles: {total_tackles}")
print(result.head())


3430
Total missed tackles: 1763
Total tackles: 1374
       gameId  playId  count  nflId  tackle  assist  forcedFumble  \
0  2022090800    1102      2  42816       1       0             0   
1  2022090800    1102      2  52492       0       0             0   
2  2022090800    1385      3  52647       1       0             0   
3  2022090800    1385      3  46085       0       1             0   
4  2022090800    1385      3  46232       0       0             0   

   pff_missedTackle  
0                 0  
1                 1  
2                 0  
3                 0  
4                 1  


In [9]:
import pandas as pd

# Load the "players.csv" file into a DataFrame
players_data = pd.read_csv('data/players.csv')

# Create a list of unique NFL IDs for players involved in a tackle
tackle_nfl_ids = result['nflId'].unique()

# Filter the players DataFrame to include only players involved in a tackle
tackling_players = players_data[players_data['nflId'].isin(tackle_nfl_ids)]

# Filter the players DataFrame to include only players not involved in a tackle
non_tackling_players = players_data[~players_data['nflId'].isin(tackle_nfl_ids)]

# Now, 'tackling_players' contains players involved in a tackle,
# and 'non_tackling_players' contains players not involved in a tackle.
print(len(tackling_players))
print(len(non_tackling_players))

661
1022


In [10]:
import pandas as pd
import concurrent.futures
import os

# Load the tackling players' NFL IDs into a list
tackling_players_nfl_ids = list(tackling_players['nflId'])

# Define the list of columns to keep
columns_to_keep = ['gameId', 'playId', 'nflId', 'x', 'y', 's', 'a', 'dir', 'time']

# Define a function to filter tracking data for a single week
def filter_tracking_data(week_num):
    tracking_file = f'data/tracking_week_{week_num}.csv'
    
    if os.path.exists(tracking_file):
        print(f'Filtering tracking data for week {week_num}...')
        
        # Load the tracking data for the week into a DataFrame
        tracking_data = pd.read_csv(tracking_file)
        
        # Filter the tracking data to include only relevant NFL IDs
        filtered_tracking_data = tracking_data[tracking_data['nflId'].isin(tackling_players_nfl_ids)]
        
        # Keep only the specified columns
        filtered_tracking_data = filtered_tracking_data[columns_to_keep]
        
        # Save the filtered tracking data to a new CSV file
        filtered_tracking_data.to_csv(f'data/filtered_tracking_week_{week_num}.csv', index=False)
        
        print(f'Filtered tracking data for week {week_num} saved.')
    else:
        print(f'Tracking data file for week {week_num} not found.')

# Use multi-threading to process each week concurrently
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    # Process weeks 1 to 9
    executor.map(filter_tracking_data, range(1, 10))

print('Filtering completed for all weeks.')


Filtering tracking data for week 1...Filtering tracking data for week 2...

Filtering tracking data for week 3...
Filtering tracking data for week 4...
Filtered tracking data for week 2 saved.
Filtering tracking data for week 5...
Filtered tracking data for week 3 saved.
Filtering tracking data for week 6...
Filtered tracking data for week 1 saved.
Filtering tracking data for week 7...
Filtered tracking data for week 4 saved.
Filtering tracking data for week 8...
Filtered tracking data for week 6 saved.
Filtering tracking data for week 9...
Filtered tracking data for week 5 saved.
Filtered tracking data for week 7 saved.
Filtered tracking data for week 8 saved.
Filtered tracking data for week 9 saved.
Filtering completed for all weeks.


In [11]:
import os
import pandas as pd

# Define the list of columns to keep
columns_to_keep = ['gameId', 'playId', 'nflId', 'x', 'y', 's', 'a', 'dir']

# Create a directory to store the filtered tracking data
filtered_tracking_dir = 'filtered_tracking_data'
if not os.path.exists(filtered_tracking_dir):
    os.makedirs(filtered_tracking_dir)

# Loop through each week (from 1 to 9)
for week_num in range(1, 10):
    input_file = f'data/filtered_tracking_week_{week_num}.csv'
    output_file = f'{filtered_tracking_dir}/filtered_tracking_week_{week_num}.csv'

    if os.path.exists(input_file):
        # Load the filtered tracking data for the week into a DataFrame
        tracking_data = pd.read_csv(input_file)

        # Filter and keep only the specified columns
        filtered_tracking_data = tracking_data[columns_to_keep]

        # Save the filtered tracking data to a new CSV file
        filtered_tracking_data.to_csv(output_file, index=False)
        
        print(f'Filtered tracking data saved for week {week_num}.')
    else:
        print(f'Filtered tracking data file for week {week_num} not found.')

print('Filtering completed for all weeks.')


Filtered tracking data saved for week 1.
Filtered tracking data saved for week 2.
Filtered tracking data saved for week 3.
Filtered tracking data saved for week 4.
Filtered tracking data saved for week 5.
Filtered tracking data saved for week 6.
Filtered tracking data saved for week 7.
Filtered tracking data saved for week 8.
Filtered tracking data saved for week 9.
Filtering completed for all weeks.


In [12]:
#find smallest time in filætered tracking data
import pandas as pd
# Read the filtered tracking data for week 1 into a DataFrame
filtered_tracking_data = pd.read_csv('data/filtered_tracking_week_1.csv')

# Find the smallest time value in the DataFrame
smallest_time = filtered_tracking_data['time'].min()

# Print the smallest time
print(f"The smallest time in filtered_tracking_week_1.csv is: {smallest_time}")


The smallest time in filtered_tracking_week_1.csv is: 2022-09-08 20:24:05.200000


In [13]:
import pandas as pd

# Load the plays data into a DataFrame
plays_data = pd.read_csv('data/plays.csv')

# Extract unique playId values from the 'result' DataFrame
unique_playIds = result['playId'].unique()

# Filter the plays data to include only playIds found in 'unique_playIds'
filtered_plays_data = plays_data[plays_data['playId'].isin(unique_playIds)]

# Keep only the specified columns
filtered_plays_data = filtered_plays_data[['gameId', 'playId', 'ballCarrierId', 'quarter', 'gameClock']]

# Now, 'filtered_plays_data' contains only the specified columns for plays with playIds found in 'result'
print(filtered_plays_data.head())


        gameId  playId  ballCarrierId  quarter gameClock
1   2022091103    3126          52457        4      7:38
2   2022091111    1148          42547        2      8:57
3   2022100212    2007          46461        3     13:12
7   2022102310      56          46377        1     15:00
13  2022103010    3127          47853        4      5:35


In [14]:
import pandas as pd

# Load the filtered plays data into a DataFrame


# Load the games data into a DataFrame
games_data = pd.read_csv('data/games.csv')

# Define a function to convert gameClock to Eastern Time
def convert_to_eastern_time(row):
    # Find the corresponding gameId in games_data
    game_info = games_data[games_data['gameId'] == row['gameId']]
    
    if not game_info.empty:
        # Extract the gameDate and gameTimeEastern from game_info
        game_date = game_info.iloc[0]['gameDate']
        game_time_eastern = game_info.iloc[0]['gameTimeEastern']
        
        # Combine gameDate and gameTimeEastern into a datetime string
        datetime_str = f"{game_date} {game_time_eastern}"
        
        # Parse the datetime string
        game_datetime = pd.to_datetime(datetime_str, format='%m/%d/%Y %H:%M:%S')
        
        # Split the gameClock to extract minutes and seconds
        minutes, seconds = map(int, row['gameClock'].split(':'))
        
        # Calculate the time difference based on the quarter
        if row['quarter'] in [1, 3]:
            game_datetime += pd.to_timedelta((15 - minutes) * 60 + (60 - seconds), unit='s')
        elif row['quarter'] == 2:
            game_datetime += pd.to_timedelta((30 - minutes) * 60 + (60 - seconds), unit='s')
        else:  # Quarter 4
            game_datetime += pd.to_timedelta((45 - minutes) * 60 + (60 - seconds), unit='s')
        
        return game_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')
    else:
        return None

# Apply the conversion function to create a new column 'easternTime'
filtered_plays_data['easternTime'] = filtered_plays_data.apply(convert_to_eastern_time, axis=1)

# Now, 'filtered_plays_data' contains the 'easternTime' column with the converted time


In [15]:
print (filtered_plays_data.head())

        gameId  playId  ballCarrierId  quarter gameClock  \
1   2022091103    3126          52457        4      7:38   
2   2022091111    1148          42547        2      8:57   
3   2022100212    2007          46461        3     13:12   
7   2022102310      56          46377        1     15:00   
13  2022103010    3127          47853        4      5:35   

                   easternTime  
1   2022-09-11 13:38:22.000000  
2   2022-09-11 16:47:03.000000  
3   2022-10-02 16:27:48.000000  
7   2022-10-23 16:26:00.000000  
13  2022-10-30 17:05:25.000000  


In [16]:
#sort filtered_plays by gameid playid and quarter
filtered_plays_data.sort_values(by=['gameId', 'playId', 'quarter'], inplace=True)
print(filtered_plays_data.head())

          gameId  playId  ballCarrierId  quarter gameClock  \
4889  2022090800      56          42489        1     15:00   
2819  2022090800      80          46076        1     14:29   
7700  2022090800     101          47857        1     13:54   
7879  2022090800     191          52494        1     11:20   
1363  2022090800     236          52536        1     10:03   

                     easternTime  
4889  2022-09-08 20:21:00.000000  
2819  2022-09-08 20:21:31.000000  
7700  2022-09-08 20:22:06.000000  
7879  2022-09-08 20:24:40.000000  
1363  2022-09-08 20:25:57.000000  


In [22]:
import pandas as pd
import os

# Load the plays data into a DataFrame
plays_data = pd.read_csv('data/plays.csv')

# Load the tackles data into a DataFrame
tackles_data = pd.read_csv('data/tackles.csv')

# Define a function to process tracking data for a given week
def process_tracking_data(week_num):
    # Load the tracking data for the week into a DataFrame
    tracking_file = f'data/tracking_week_{week_num}.csv'
    
    if os.path.exists(tracking_file):
        print(f'Processing tracking data for week {week_num}...')
        tracking_data = pd.read_csv(tracking_file)
        
        # Step 1: Find rows with "out_of_bounds" in tracking data
        out_of_bounds_rows = tracking_data[tracking_data['event'] == 'out_of_bounds']
        
        # Step 2: Match the corresponding play using gameId and playId
        merged_data = pd.merge(out_of_bounds_rows, plays_data, on=['gameId', 'playId'], how='inner')
        
        # Step 3: Find the matching tackle information (if it exists)
        merged_data_with_tackle = pd.merge(merged_data, tackles_data, on=['gameId', 'playId'], how='left')
        
        # Now, you can further analyze or save the data for this week as needed
        
        print(f'Processing completed for week {week_num}.')
    else:
        print(f'Tracking data file for week {week_num} not found.')

# Use a loop to process each week's tracking data
for week_num in range(1, 10):
    process_tracking_data(week_num)


Processing tracking data for week 1...
Processing completed for week 1.
Processing tracking data for week 2...
Processing completed for week 2.
Processing tracking data for week 3...
Processing completed for week 3.
Processing tracking data for week 4...
Processing completed for week 4.
Processing tracking data for week 5...
Processing completed for week 5.
Processing tracking data for week 6...
Processing completed for week 6.
Processing tracking data for week 7...
Processing completed for week 7.
Processing tracking data for week 8...
Processing completed for week 8.
Processing tracking data for week 9...
Processing completed for week 9.


In [25]:
import pandas as pd
import os

# Load the plays data into a DataFrame
plays_data = pd.read_csv('data/plays.csv')

# Load the tackles data into a DataFrame
tackles_data = pd.read_csv('data/tackles.csv')

# Initialize a global counter for out_of_bounds events with a tackle
out_of_bounds_with_tackle_count = 0

# Define a function to process tracking data for a given week
def process_tracking_data(week_num):
    global out_of_bounds_with_tackle_count  # Declare the variable as global
    
    # Load the tracking data for the week into a DataFrame
    tracking_file = f'data/tracking_week_{week_num}.csv'
    
    if os.path.exists(tracking_file):
        print(f'Processing tracking data for week {week_num}...')
        tracking_data = pd.read_csv(tracking_file)
        
        # Step 1: Find rows with "out_of_bounds" in tracking data
        out_of_bounds_rows = tracking_data[tracking_data['event'] == 'out_of_bounds']
        
        # Step 2: Match the corresponding play using gameId and playId
        merged_data = pd.merge(out_of_bounds_rows, plays_data, on=['gameId', 'playId'], how='inner')
        
        # Step 3: Find the matching tackle information (if it exists)
        merged_data_with_tackle = pd.merge(merged_data, tackles_data, on=['gameId', 'playId'], how='left')
        
        # Count the occurrences of "out_of_bounds" events with a tackle
        out_of_bounds_with_tackle_count += len(merged_data_with_tackle)
        
        print(f'Processing completed for week {week_num}.')
    else:
        print(f'Tracking data file for week {week_num} not found.')

# Use a loop to process each week's tracking data
for week_num in range(1, 10):
    process_tracking_data(week_num)

# Print the total count of "out_of_bounds" events with a tackle
print(f'Total "out_of_bounds" events with a tackle: {out_of_bounds_with_tackle_count}')


Processing tracking data for week 1...
Processing completed for week 1.
Processing tracking data for week 2...
Processing completed for week 2.
Processing tracking data for week 3...
Processing completed for week 3.
Processing tracking data for week 4...
Processing completed for week 4.
Processing tracking data for week 5...
Processing completed for week 5.
Processing tracking data for week 6...
Processing completed for week 6.
Processing tracking data for week 7...
Processing completed for week 7.
Processing tracking data for week 8...
Processing completed for week 8.
Processing tracking data for week 9...
Processing completed for week 9.
Total "out_of_bounds" events with a tackle: 51749


In [26]:
import pandas as pd
import os

# Initialize a global counter for out_of_bounds events
out_of_bounds_count = 0

# Define a function to process tracking data for a given week
def process_tracking_data(week_num):
    global out_of_bounds_count  # Declare the variable as global
    
    # Load the tracking data for the week into a DataFrame
    tracking_file = f'data/tracking_week_{week_num}.csv'
    
    if os.path.exists(tracking_file):
        print(f'Processing tracking data for week {week_num}...')
        tracking_data = pd.read_csv(tracking_file)
        
        # Step 1: Count occurrences of "out_of_bounds" in the 'event' column
        out_of_bounds_week_count = len(tracking_data[tracking_data['event'] == 'out_of_bounds'])
        
        # Accumulate the count to the global total
        out_of_bounds_count += out_of_bounds_week_count
        
        print(f'Processing completed for week {week_num}.')
    else:
        print(f'Tracking data file for week {week_num} not found.')

# Use a loop to process each week's tracking data
for week_num in range(1, 10):
    process_tracking_data(week_num)

# Print the total count of "out_of_bounds" events
print(f'Total "out_of_bounds" events across weeks 1 to 9: {out_of_bounds_count}')


Processing tracking data for week 1...
Processing completed for week 1.
Processing tracking data for week 2...
Processing completed for week 2.
Processing tracking data for week 3...
Processing completed for week 3.
Processing tracking data for week 4...
Processing completed for week 4.
Processing tracking data for week 5...
Processing completed for week 5.
Processing tracking data for week 6...
Processing completed for week 6.
Processing tracking data for week 7...
Processing completed for week 7.
Processing tracking data for week 8...
Processing completed for week 8.
Processing tracking data for week 9...
Processing completed for week 9.
Total "out_of_bounds" events across weeks 1 to 9: 41629


In [28]:
import pandas as pd

# Load the tackles data into a DataFrame
tackles_data = pd.read_csv('data/tackles.csv')

# Initialize counters
tackle_count = 0
missed_tackle_count = 0
missed_tackle_with_out_of_bounds_count = 0

# Step 1: Identify missed tackles
missed_tackle_plays = tackles_data.groupby(['playId', 'gameId']).filter(lambda x: x['pff_missedTackle'].sum() == len(x))

# Step 2: Check if missed tackles correspond to "out_of_bounds" events
for idx, combo in missed_tackle_plays.groupby(['playId', 'gameId']).size().reset_index().iterrows():
    playId = combo['playId']
    gameId = combo['gameId']
    
    # Search tracking data for "out_of_bounds" events for the same player
    tracking_data_file = f'data/tracking_week_{week_num}.csv'  # Replace week_num with the appropriate week
    tracking_data = pd.read_csv(tracking_data_file)
    
    player_events = tracking_data[(tracking_data['playId'] == playId) & (tracking_data['gameId'] == gameId) & (tracking_data['event'] == 'out_of_bounds')]
    
    if not player_events.empty:
        missed_tackle_with_out_of_bounds_count += 1
    else:
        missed_tackle_count += 1

# Step 3: Analyze the results
tackle_count = len(tackles_data) - missed_tackle_count  # Total tackles
print(f'Total Tackles: {tackle_count}')
print(f'Total Missed Tackles: {missed_tackle_count}')
print(f'Missed Tackles with Out of Bounds: {missed_tackle_with_out_of_bounds_count}')


Number of combos with out_of_bounds: 2
Number of combos without out_of_bounds: 128


In [30]:
import pandas as pd

# Load the tackles data into a DataFrame
tackles_data = pd.read_csv('data/tackles.csv')

# Initialize counters
tackle_count = 0
missed_tackle_count = 0
missed_tackle_with_out_of_bounds_count = 0

# Iterate through weeks 1 to 9
for week_num in range(1, 10):
    # Step 1: Identify missed tackles
    missed_tackle_plays = tackles_data.groupby(['playId', 'gameId']).filter(lambda x: x['pff_missedTackle'].sum() == len(x))

    # Step 2: Check if missed tackles correspond to "out_of_bounds" events
    for idx, combo in missed_tackle_plays.groupby(['playId', 'gameId']).size().reset_index().iterrows():
        playId = combo['playId']
        gameId = combo['gameId']

        # Search tracking data for "out_of_bounds" events for the same player
        tracking_data_file = f'data/tracking_week_{week_num}.csv'
        tracking_data = pd.read_csv(tracking_data_file)

        player_events = tracking_data[(tracking_data['playId'] == playId) & (tracking_data['gameId'] == gameId) & (tracking_data['event'] == 'out_of_bounds')]

        if not player_events.empty:
            missed_tackle_with_out_of_bounds_count += 1
        else:
            missed_tackle_count += 1

# Step 3: Analyze the results
tackle_count = len(tackles_data) - missed_tackle_count  # Total tackles
print(f'Total Tackles: {tackle_count}')
print(f'Total Missed Tackles: {missed_tackle_count}')
print(f'Missed Tackles with Out of Bounds: {missed_tackle_with_out_of_bounds_count}')


Total Tackles: 16268
Total Missed Tackles: 1158
Missed Tackles with Out of Bounds: 12


In [6]:
import pandas as pd
tackles = pd.read_csv('data/tackles.csv')
# Assuming you have already loaded the "tackles" DataFrame
# Filter the DataFrame to include rows with "tackle" and "pff_missedTackle" but not "assist" or "forcedFumble"
filtered_tackles = tackles[~((tackles['tackle'] == 1) & (tackles['pff_missedTackle'] == 1))]
filtred_tackles = filtered_tackles[~((filtered_tackles['assist'] == 1) & (filtered_tackles['forcedFumble'] == 1))]
# Now "filtered_tackles" contains the rows meeting your criteria
#create csv file with filtered tackles
filtered_tackles.to_csv('data/filtered_tackles.csv', index=False)



          gameId  playId  nflId  tackle  assist  forcedFumble  \
219   2022091805    2361  52444       1       0             0   
455   2022091102     189  52571       1       0             0   
696   2022091105    2742  53450       1       0             0   
935   2022091110     750  52546       1       0             0   
1213  2022091200    3723  52435       1       0             0   

      pff_missedTackle  
219                  1  
455                  1  
696                  1  
935                  1  
1213                 1  
